In [89]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv
import os

In [108]:
load_dotenv()

True

In [6]:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
url = base_url + "/api/3/action/package_show"
params = { "id": "daily-shelter-overnight-service-occupancy-capacity"}
package = requests.get(url, params = params).json()

In [71]:
df = None
for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"] and str(resource["created"]).startswith(str(datetime.now().year)):
        url = base_url + "/datastore/dump/" + resource["id"]
        data = requests.get(url).text
        df = pd.read_csv(StringIO(data))
df.drop([
    "_id", 
    "OCCUPANCY_DATE", 
    "ORGANIZATION_ID", 
    "ORGANIZATION_NAME", 
    "SHELTER_ID", 
    "SHELTER_GROUP", 
    "PROGRAM_ID", 
    "PROGRAM_NAME", 
    "PROGRAM_MODEL", 
    "OCCUPIED_BEDS", 
    "UNAVAILABLE_BEDS", 
    "CAPACITY_ACTUAL_ROOM", 
    "CAPACITY_FUNDING_ROOM", 
    "OCCUPIED_ROOMS", 
    "UNOCCUPIED_ROOMS", 
    "UNAVAILABLE_ROOMS", 
    "OCCUPANCY_RATE_BEDS", 
    "OCCUPANCY_RATE_ROOMS", 
    "OVERNIGHT_SERVICE_TYPE", 
    "PROGRAM_AREA", 
    "SERVICE_USER_COUNT", 
    "CAPACITY_TYPE", 
    "CAPACITY_ACTUAL_BED", 
    "CAPACITY_FUNDING_BED"
    ], axis=1, inplace=True)

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6136 entries, 9 to 48793
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LOCATION_ID           6136 non-null   float64
 1   LOCATION_NAME         6133 non-null   object 
 2   LOCATION_ADDRESS      5765 non-null   object 
 3   LOCATION_POSTAL_CODE  5777 non-null   object 
 4   LOCATION_CITY         5772 non-null   object 
 5   LOCATION_PROVINCE     5772 non-null   object 
 6   SECTOR                6136 non-null   object 
 7   UNOCCUPIED_BEDS       6136 non-null   float64
dtypes: float64(2), object(6)
memory usage: 431.4+ KB


In [80]:
df["UNOCCUPIED_BEDS"].fillna(0)
df = df[df["UNOCCUPIED_BEDS"] > 0]

In [157]:
try:
    uri = os.getenv("MONGO_CONNECTION_STRING")
    client = MongoClient(uri)
    database = client["Info"]
    collection = database["LocationCache"]
    results = list(collection.find({}, {"_id": 0}))
    client.close()
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

In [159]:
base_geocode_url = "https://geocode.googleapis.com/v4beta/geocode/address/"
GEOCODE_MAPS_API_KEY = os.getenv("GEOCODE_MAPS_API_KEY")
try:
    uri = os.getenv("MONGO_CONNECTION_STRING")
    client = MongoClient(uri)
    database = client["Info"]
    collection = database["LocationCache"]
    for index, row in df.iterrows():
        base_address = row["LOCATION_ADDRESS"]
        if base_address in [list(e.keys())[0] for e in results]:
            print(".", end="")
        else:
            address = str(base_address).replace(" ", "+")
            url = base_geocode_url + address + "?key=" + GEOCODE_MAPS_API_KEY
            data = requests.get(url = url).json()
            obj = {base_address: {
                "name": row["LOCATION_NAME"],
                "id": row["LOCATION_ID"],
                "postal_code": row["LOCATION_POSTAL_CODE"],
                "city": row["LOCATION_CITY"],
                "type": "shelter",
                "lat": data["results"][0]["location"]["latitude"],
                "long": data["results"][0]["location"]["longitude"]
            }}
            print(obj)
            results.append(obj)
            collection.insert_one(obj)
    client.close()
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

...........................................................................................................................................{'69 Fraser Ave': {'name': 'St. Felix Centre 69 Fraser Ave. Respite ', 'id': 1148.0, 'postal_code': 'M6K 1E9', 'city': 'Toronto', 'type': 'shelter', 'lat': 43.637866599999995, 'long': -79.423853}}
................................................{'135 Sherbourne St': {'name': 'Maxwell Meighen Centre', 'id': 1057.0, 'postal_code': 'M5A 2R5', 'city': 'Toronto', 'type': 'shelter', 'lat': 43.655291899999995, 'long': -79.3690019}}
..{'100 Lippincott St': {'name': 'COSTI Reception Centre', 'id': 1051.0, 'postal_code': 'M5S 2P1', 'city': 'Toronto', 'type': 'shelter', 'lat': 43.6575621, 'long': -79.4073935}}
.................{'3576 St Clair Ave E': {'name': 'HFS St. Clair Ave E Shelter', 'id': 1031.0, 'postal_code': 'M1K 1M2', 'city': 'Toronto', 'type': 'shelter', 'lat': 43.717528, 'long': -79.2581094}}
................{'674 Dundas St W': {'name': "TSSS Wome

In [160]:
locations = {}
for e in results:
    e_add = list(e.keys())[0]
    locations[e_add] = e[e_add]

In [161]:
try:
    uri = os.getenv("MONGO_CONNECTION_STRING")
    client = MongoClient(uri)
    database = client["Info"]
    collection = database["ShelterInfo"]
    for index, row in df.iterrows():
        base_address = row["LOCATION_ADDRESS"]
        obj = {
            "address": base_address,
            "unoccupied_beds": row["UNOCCUPIED_BEDS"],
            "sector": row["SECTOR"],
            "name": row["LOCATION_NAME"],
            "id": row["LOCATION_ID"],
            "postal_code": row["LOCATION_POSTAL_CODE"],
            "city": row["LOCATION_CITY"],
            "lat": locations[base_address]["lat"],
            "long": locations[base_address]["long"]
        }
        print(obj)
        collection.insert_one(obj)
    client.close()
        
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

{'address': '136 Spadina Rd', 'unoccupied_beds': 4.0, 'sector': 'Mixed Adult', 'name': 'Christie Ossington - Spadina Rd', 'id': 1541.0, 'postal_code': 'M5R 2T8', 'city': 'Toronto', 'lat': 43.6731171, 'long': -79.4066679}
{'address': '885 Scarborough Golf Club Road', 'unoccupied_beds': 27.0, 'sector': 'Mixed Adult', 'name': '885 Scarborough Golf Club Road', 'id': 1561.0, 'postal_code': 'M1G 1J6', 'city': 'Toronto', 'lat': 43.77281, 'long': -79.21923629999999}
{'address': '1651 Sheppard Ave W', 'unoccupied_beds': 4.0, 'sector': 'Men', 'name': 'TSSS Downsview Dells', 'id': 1009.0, 'postal_code': 'M3M 2X4', 'city': 'Toronto', 'lat': 43.740637199999995, 'long': -79.49416029999999}
{'address': '38 Bathurst St', 'unoccupied_beds': 7.0, 'sector': 'Men', 'name': 'TSSS Fort York Residence', 'id': 1011.0, 'postal_code': 'M5V 3W3', 'city': 'Toronto', 'lat': 43.641175399999995, 'long': -79.40251599999999}
{'address': '38 Bathurst St', 'unoccupied_beds': 2.0, 'sector': 'Men', 'name': 'TSSS Fort York